In [2]:
import time
import warnings
import gymnasium as gym
import pandas as pd
import matplotlib.pyplot as plt

from environment.env import ShogiEnv
from agent.shogi_agent import ShogiAgent

warnings.filterwarnings("ignore")

gym.register(id="Shogi-v0", entry_point="environment.env:ShogiEnv")
env: ShogiEnv = gym.make("Shogi-v0")
agent = ShogiAgent()
agent.get_model("models/shogi-agent.pth")

In [92]:
import chess
import chess.engine
import shogi

# Define the path to the Fairy-Stockfish binary
FAIRY_STOCKFISH_PATH = "fairy-stockfish-largeboard_x86-64.exe"

# Initialize the Fairy-Stockfish engine
fairy_stockfish = chess.engine.SimpleEngine.popen_uci(FAIRY_STOCKFISH_PATH)

fairy_stockfish.configure({"UCI_Variant": "shogi"})

# Set time limits for the engines
time_limit = chess.engine.Limit(time=2.0)

# Initialize the environment (assuming env is defined and reset correctly)
env.reset()

# Play a match
while not env.board.is_game_over():
    print(env.board)

    # Fairy-Stockfish move
    result = fairy_stockfish.play(env.board, time_limit)
    env.board.push(result.move)
    print(f"Fairy-Stockfish plays: {result.move}")

    if env.board.is_game_over():
        break

    # Custom engine move
    custom_engine_move = agent.select_best_action(env)
    env.board.push(chess.Move.from_uci(str(custom_engine_move[0])))
    print(f"Custom engine plays: {custom_engine_move[0]}")

    if env.board.is_game_over():
        break

# Print the result
result = env.board.result()
print("Game over!")
print("Result:", result)

# Close the engines
fairy_stockfish.quit()


EngineError: cannot set UCI_Variant which is automatically managed